In [ ]:
import pathlib

import marimo as mo
import polars as pl

In [ ]:
hathi_data_dir = pathlib.Path("data/hathi/updates")

field_list = (hathi_data_dir / "hathi_field_list.txt").open().read().split()
field_list

In [ ]:
# load most recent full dataset with specified field list; treat as TSV

df = (
    pl.scan_csv(
        hathi_data_dir / "hathi_full_20250701.txt.gz",
        has_header=False,
        new_columns=field_list,
        separator="\t",
        quote_char=None,  # do not treat " as escape character, since it is used in some content
        encoding="utf8",
    )
    .select(
        ["htid", "access", "rights", "collection_code", "access_profile_code"]
    )
    .collect()
)

df.head(10)

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"htid\":\"mdp.39015018415946\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"MIU\",\"access_profile_code\":\"google\"},{\"htid\":\"mdp.39015066356547\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"MIU\",\"access_profile_code\":\"google\"},{\"htid\":\"mdp.39015066356406\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"MIU\",\"access_profile_code\":\"google\"},{\"htid\":\"mdp.39015066356695\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"MIU\",\"access_profile_code\":\"google\"},{\"htid\":\"mdp.39015066356554\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"MIU\",\"access_profile_code\":\"google\"},{\"htid\":\"uc1.$b759626\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"NRLF\",\"access_profile_code\":\"google\"},{\"htid\":\"uc1.$b759627\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"NRLF\",\"access_profile_code\":\"google\"},{\"htid\":\"uc1.$b759628\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"NRLF\",\"access_profile_code\":\"google\"},{\"htid\":\"mdp.39015033913115\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"MIU\",\"access_profile_code\":\"google\"},{\"htid\":\"mdp.39015061455294\",\"access\":\"deny\",\"rights\":\"ic\",\"collection_code\":\"MIU\",\"access_profile_code\":\"google\"}]"' data-total-rows='10' data-total-columns='5' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["htid", ["string", "str"]], ["access", ["string", "str"]], ["rights", ["string", "str"]], ["collection_code", ["string", "str"]], ["access_profile_code", ["string", "str"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
total_ht_vols = df.height

print(f"{total_ht_vols:,} total volumes")

18,920,632 total volumes


In [ ]:
df["rights"].value_counts()

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"rights\":\"nobody\",\"count\":1145},{\"rights\":\"icus\",\"count\":69697},{\"rights\":\"cc-by-sa-3.0\",\"count\":94},{\"rights\":\"cc-by-4.0\",\"count\":27460},{\"rights\":\"pd-pvt\",\"count\":1532},{\"rights\":\"cc-by-nc-nd-4.0\",\"count\":21010},{\"rights\":\"ic-world\",\"count\":7697},{\"rights\":\"cc-by-3.0\",\"count\":882},{\"rights\":\"cc-by-nc-sa-4.0\",\"count\":1666},{\"rights\":\"cc-by-nd-4.0\",\"count\":881}]"' data-total-rows='23' data-total-columns='2' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["rights", ["string", "str"]], ["count", ["integer", "u32"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
df["access"].value_counts()

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"access\":\"deny\",\"count\":10999205},{\"access\":\"allow\",\"count\":7921427}]"' data-total-rows='2' data-total-columns='2' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["access", ["string", "str"]], ["count", ["integer", "u32"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
df["access_profile_code"].value_counts()

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"access_profile_code\":\"page+lowres\",\"count\":21218},{\"access_profile_code\":\"google\",\"count\":17759424},{\"access_profile_code\":\"open\",\"count\":1138679},{\"access_profile_code\":\"page\",\"count\":1311}]"' data-total-rows='4' data-total-columns='2' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["access_profile_code", ["string", "str"]], ["count", ["integer", "u32"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

Load PPA data so we can get counts for items in PPA changing as well as all of HT.

In [ ]:
# load PPA work-level metadata and limit to HathiTrust content
ppa_ht_df = pl.read_csv("data/ppa/ppa_work_metadata.csv").filter(
    pl.col("ppa_source").eq("HathiTrust")
)
# get the total number of unique HT volumes in PPA; use unique since a few excerpts are from the same volume
ppa_ht_total = ppa_ht_df["ppa_source_id"].unique().len()

ppa_ht_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"ppa_work_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"ppa_source_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"ppa_cluster_id\":\"aeu.ark:/13960/t0ft9gj3x\",\"ppa_work_title\":\"Treatise on French pronunciation and genders\",\"ppa_work_author\":\"Lafont, J. B. Angelvy\",\"ppa_work_year\":1864,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Linguistic']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t1hh7pc2q\",\"ppa_source_id\":\"aeu.ark:/13960/t1hh7pc2q\",\"ppa_cluster_id\":\"aeu.ark:/13960/t1hh7pc2q\",\"ppa_work_title\":\"An essay on the elements, accents, & prosody, of the English language\",\"ppa_work_author\":\"Odell, Jonathan, 1737-1818\",\"ppa_work_year\":1805,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary', 'Typographically Unique']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t1pg22p71-p126\",\"ppa_source_id\":\"aeu.ark:/13960/t1pg22p71\",\"ppa_cluster_id\":\"aeu.ark:/13960/t1pg22p71-p126\",\"ppa_work_title\":\"On the Galliambic Metre\",\"ppa_work_author\":\"Allen, [Charles] Grant [Blairfindie].\",\"ppa_work_year\":1892,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary', 'Original Bibliography', 'Typographically Unique']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t3514pf3k\",\"ppa_source_id\":\"aeu.ark:/13960/t3514pf3k\",\"ppa_cluster_id\":\"aeu.ark:/13960/t3514pf3k\",\"ppa_work_title\":\"An elementary Greek grammar\",\"ppa_work_author\":\"Goodwin, William W. (William Watson), 1831-1912\",\"ppa_work_year\":1879,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Linguistic']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t3jw96h3m\",\"ppa_source_id\":\"aeu.ark:/13960/t3jw96h3m\",\"ppa_cluster_id\":\"aeu.ark:/13960/t3jw96h3m\",\"ppa_work_title\":\"Metra horatiana, or, The metrical systems of Horace\",\"ppa_work_author\":\"Lundy, F. J.\",\"ppa_work_year\":1838,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t5s75qs8g\",\"ppa_source_id\":\"aeu.ark:/13960/t5s75qs8g\",\"ppa_cluster_id\":\"aeu.ark:/13960/t5s75qs8g\",\"ppa_work_title\":\"Pentasyllabic endings in the Latin hexameter with particular reference to the verse of Lucretius\",\"ppa_work_author\":\"Alexander, William Hardy, 1878-1962\",\"ppa_work_year\":1900,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t7qn72r42\",\"ppa_source_id\":\"aeu.ark:/13960/t7qn72r42\",\"ppa_cluster_id\":\"aeu.ark:/13960/t7qn72r42\",\"ppa_work_title\":\"Brief rules of the Latin prosody\",\"ppa_work_author\":\"Cochran, William, 1757-1833\",\"ppa_work_year\":1899,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"},{\"ppa_work_id\":\"aeu.ark:/13960/t8v98vx29\",\"ppa_source_id\":\"aeu.ark:/13960/t8v98vx29\",\"ppa_cluster_id\":\"aeu.ark:/13960/t8v98vx29\",\"ppa_work_title\":\"Introduction to the science of language\",\"ppa_work_author\":\"Sayce, A. H. (Archibald Henry), 1845-1933\",\"ppa_work_year\":1900,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Linguistic']\"},{\"ppa_work_id\":\"bc.ark:/13960/t2p63fq11\",\"ppa_source_id\":\"bc.ark:/13960/t2p63fq11\",\"ppa_cluster_id\":\"bc.ark:/13960/t2p63fq11\",\"ppa_work_title\":\"An introduction to Latin elegiac verse composition,\",\"ppa_work_author\":\"Lupton, J. H. (Joseph Hirst), 1836-1905\",\"ppa_work_year\":1913,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"},{\"ppa_work_id\":\"chi.084960397\",\"ppa_source_id\":\"chi.084960397\",\"ppa_cluster_id\":\"chi.084960397\",\"ppa_work_title\":\"A study of the principal Latin rhythms, other than the hexameter ...\",\"ppa_work_author\":\"Clough, Joseph W.\",\"ppa_work_year\":1879,\"ppa_source\":\"HathiTrust\",\"ppa_collections\":\"['Literary']\"}]"' data-total-rows='5539' data-total-columns='8' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["ppa_work_id", ["string", "str"]], ["ppa_source_id", ["string", "str"]], ["ppa_cluster_id", ["string", "st

In [ ]:
import datetime
from polars.exceptions import NoDataError

update_data = []

for update_file in hathi_data_dir.glob("hathi_upd_*.txt.gz"):
    update_date_str = update_file.stem.rsplit("_")[-1].replace(".txt", "")
    update_date = datetime.datetime.strptime(update_date_str, "%Y%m%d").date()

    # at least one file is actually empty; handle the error and skip
    try:
        update_df = (
            pl.scan_csv(
                update_file,
                has_header=False,
                new_columns=field_list,
                separator="\t",
                quote_char=None,  # do not treat " as escape character, since it is used in smoe content
                encoding="utf8",
            )
            .select(["htid"])  # , "access", "rights", "collection_code"])
            .collect()
        )
        # join with ppa data so we can count # ppa volumes that changed
        ppa_updates_df = update_df.join(
            ppa_ht_df, left_on="htid", right_on="ppa_source_id", how="inner"
        )

        update_data.append(
            {
                "date": update_date,
                "num_updated": update_df.height,
                "ppa_updated": ppa_updates_df.height,
            }
        )
    except NoDataError as err:
        print(f"Error parsing {update_file.name} : {err}")

Error parsing hathi_upd_20250509.txt.gz : empty CSV


In [ ]:
update_data_df = pl.from_dicts(update_data)
update_data_df = (
    # calculate percentages for all of hathitrust and then all of ppa
    update_data_df.with_columns(
        pct_updated=pl.col("num_updated").truediv(total_ht_vols),
        pct_ppa_updated=pl.col("ppa_updated").truediv(ppa_ht_total),
    )
    .cast({"date": pl.Date})
    .sort("date")
)
update_data_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"date\":\"2025-05-01\",\"num_updated\":459707,\"ppa_updated\":39,\"pct_updated\":0.024296598549139372,\"pct_ppa_updated\":0.007269338303821062},{\"date\":\"2025-05-02\",\"num_updated\":377472,\"ppa_updated\":65,\"pct_updated\":0.01995028495876882,\"pct_ppa_updated\":0.012115563839701771},{\"date\":\"2025-05-03\",\"num_updated\":423831,\"ppa_updated\":116,\"pct_updated\":0.022400467383964763,\"pct_ppa_updated\":0.021621621621621623},{\"date\":\"2025-05-04\",\"num_updated\":265070,\"ppa_updated\":135,\"pct_updated\":0.014009574310202746,\"pct_ppa_updated\":0.025163094128611372},{\"date\":\"2025-05-05\",\"num_updated\":420486,\"ppa_updated\":111,\"pct_updated\":0.022223676249292307,\"pct_ppa_updated\":0.020689655172413793},{\"date\":\"2025-05-06\",\"num_updated\":457121,\"ppa_updated\":52,\"pct_updated\":0.02415992235354506,\"pct_ppa_updated\":0.009692451071761417},{\"date\":\"2025-05-07\",\"num_updated\":118717,\"ppa_updated\":10,\"pct_updated\":0.006274473283979097,\"pct_ppa_updated\":0.001863932898415657},{\"date\":\"2025-05-08\",\"num_updated\":403917,\"ppa_updated\":23,\"pct_updated\":0.021347965543645687,\"pct_ppa_updated\":0.004287045666356011},{\"date\":\"2025-05-10\",\"num_updated\":675112,\"ppa_updated\":88,\"pct_updated\":0.03568126054140263,\"pct_ppa_updated\":0.01640260950605778},{\"date\":\"2025-05-11\",\"num_updated\":440304,\"ppa_updated\":51,\"pct_updated\":0.023271104263324818,\"pct_ppa_updated\":0.009506057781919852}]"' data-total-rows='70' data-total-columns='5' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["date", ["date", "date"]], ["num_updated", ["integer", "i64"]], ["ppa_updated", ["integer", "i64"]], ["pct_updated", ["number", "f64"]], ["pct_ppa_updated", ["number", "f64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
# save totals calculated from these files
update_data_df.write_csv("data/hathi_update_counts.csv")

In [ ]:
import altair as alt

# get date range for the data
earliest = update_data_df["date"].min()
latest = update_data_df["date"].max()

num_chart = (
    alt.Chart(update_data_df)
    .mark_bar(width=10, color="#4661ac")
    .encode(
        x=alt.X("date", title="Date").axis(
            format="%B %d", tickCount="week"
        ),  # suppress labels since will be combined
        y=alt.Y("num_updated", title="# updated"),
    )
).properties(
    width=840,
    height=100,
    # title=f"Updated volumes in all of HathiTrust, {earliest.strftime('%B %d')} to {latest.strftime('%B %d %Y')}",
)


# can we set y-domain to force the axes to align?
pct_chart = (
    num_chart.mark_bar(width=10, color="#ff7f0e", opacity=0.5)
    .encode(
        x=alt.X("date", title="Date").axis(format="%B %d", tickCount="week"),
        y=alt.Y("pct_updated", title="% updated"),
    )
    .properties(
        height=100,
    )
)


#     )

num_chart.save("figures/hathitrust_changes_countonly.pdf")

num_chart
# can we combine and fix the scale so % is one side and count is the other?
# right now they don't match exactly
# combined_chart = (
#     (num_chart + pct_chart)
#     .properties(
#         title=f"Updated volumes in all of HathiTrust, {earliest.strftime('%B %d')} to {latest.strftime('%B %d %Y')}"
#     )
#     .resolve_scale(x="shared", y="independent")
# )


# combined_chart = (
#     (num_chart & pct_chart)
#     .properties(
#         title=f"Updated volumes in all of HathiTrust, {earliest.strftime('%B %d')} to {latest.strftime('%B %d %Y')}"
#     )
#     .resolve_scale(x="shared")
# )
# combined_chart.save("figures/hathitrust_changes.pdf")

# combined_chart

In [ ]:
mo.md(
    f"Updated volumes in all of HathiTrust, {earliest.strftime('%B %d')} to {latest.strftime('%B %d %Y')}"
)

Updated volumes in all of HathiTrust, May 01 to July 10 2025

What is the largest number of updates in a single day during this time period?

In [ ]:
update_data_df.sort("num_updated", descending=True).head(5)

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"date\":\"2025-05-15\",\"num_updated\":846329,\"ppa_updated\":27,\"pct_updated\":0.0447304825758463,\"pct_ppa_updated\":0.005032618825722274},{\"date\":\"2025-05-30\",\"num_updated\":687959,\"ppa_updated\":34,\"pct_updated\":0.0363602547737306,\"pct_ppa_updated\":0.006337371854613234},{\"date\":\"2025-05-10\",\"num_updated\":675112,\"ppa_updated\":88,\"pct_updated\":0.03568126054140263,\"pct_ppa_updated\":0.01640260950605778},{\"date\":\"2025-06-24\",\"num_updated\":647451,\"ppa_updated\":42,\"pct_updated\":0.03421931149022929,\"pct_ppa_updated\":0.00782851817334576},{\"date\":\"2025-06-01\",\"num_updated\":615189,\"ppa_updated\":43,\"pct_updated\":0.032514188743800944,\"pct_ppa_updated\":0.008014911463187325}]"' data-total-rows='5' data-total-columns='5' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["date", ["date", "date"]], ["num_updated", ["integer", "i64"]], ["ppa_updated", ["integer", "i64"]], ["pct_updated", ["number", "f64"]], ["pct_ppa_updated", ["number", "f64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
max_update = update_data_df.sort("num_updated", descending=True).row(
    0, named=True
)

max_num_updated = max_update["num_updated"]
max_pct_updated = max_update["pct_updated"]
max_updates_day = max_update["date"]
mo.md(
    f"Largest number updated: {max_num_updated:,} ({max_pct_updated * 100:.1f}%) on {max_updates_day}"
)

Largest number updated: 846,329 (4.5%) on 2025-05-15

What is the average daily change across this time period?

In [ ]:
avg_num_updated = update_data_df["num_updated"].mean()
avg_pct_updated = update_data_df["pct_updated"].mean()
mo.md(
    f"Average daily update for this period is {int(avg_num_updated):,} ({avg_pct_updated * 100:.1f}%)"
)

Average daily update for this period is 348,392 (1.8%)

In [ ]:
# ppa charts equivalent to above

ppa_num_chart = (
    alt.Chart(update_data_df)
    .mark_bar(width=10, color="#f05b69")
    .encode(
        x=alt.X("date", title="").axis(
            format="%B %d",
            tickCount="week",  # labels=False to suppress for combination
        ),  # suppress labels since displayed together
        y=alt.Y("ppa_updated", title="# volumes updated"),
    )
    .properties(
        height=100,
        width=840,
        # title=f"Updates to PPA volumes in HathiTrust, {earliest.strftime('%B %d')} to {latest.strftime('%B %d %Y')}",
    )
)


ppa_pct_chart = (
    num_chart.mark_bar(width=10, color="#57c4c4")
    .encode(
        x=alt.X("date", title="Date").axis(format="%B %d", tickCount="week"),
        y=alt.Y("pct_ppa_updated", title="% updated"),
    )
    .properties(height=100)
)

ppa_combined_chart = (
    (ppa_num_chart & ppa_pct_chart)
    .properties(
        title=f"Updates to PPA volumes in HathiTrust, {earliest.strftime('%B %d')} to {latest.strftime('%B %d %Y')}"
    )
    .resolve_scale(x="shared")
)
# ppa_combined_chart.save("figures/ppa_hathitrust_changes.pdf")
ppa_combined_chart

In [ ]:
# num_chart.save("figures/hathitrust_changes_countonly.pdf")
ppa_num_chart.save("figures/ppa_hathitrust_changes_countonly.pdf")
ppa_num_chart

In [ ]:
ppa_ht_numchart = (
    alt.vconcat(ppa_num_chart, num_chart)
    .resolve_axis(x="shared")
    .properties(
        title=f"Updates to PPA HathiTrust volumes and all of HathiTrust, {earliest.strftime('%B %d')} to {latest.strftime('%B %d %Y')}"
    )
)

# saving this chart fails for smoe reason
# ppa_ht_numchart.save("figures/ppa_and_hathitrust_changes.pdf")

ppa_ht_numchart

## deletions from public domain dataset

In [ ]:
hathi_deletion_dir = pathlib.Path("data/hathi/deletions")

deletion_count = []

start_str = "===BEGIN ID LIST==="
end_str = "===END ID LIST==="

for deletion_email in hathi_deletion_dir.glob("*.txt"):
    deletion_date = datetime.datetime.strptime(
        deletion_email.stem, "%Y-%m-%d"
    ).date()
    contents = deletion_email.open().read()
    # use begin/end strings to isolate content of interest
    deleted_id_list = contents.split(start_str)[1].split(end_str)[0]
    # split on newlines and filter out any empty strings
    deleted_ids = [
        id for id in deleted_id_list.split("\n") if id.strip() != ""
    ]
    deletion_count.append({"date": deletion_date, "count": len(deleted_ids)})

deletion_df = (
    pl.from_dicts(deletion_count)
    .sort("date")
    .with_columns(percent=pl.col("count").truediv(total_ht_vols))
)
deletion_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"date\":\"2025-01-01\",\"count\":111,\"percent\":5.866611643839381e-06},{\"date\":\"2025-02-01\",\"count\":191,\"percent\":1.0094800215975872e-05},{\"date\":\"2025-03-01\",\"count\":81,\"percent\":4.281040929288197e-06},{\"date\":\"2025-04-01\",\"count\":182,\"percent\":9.619129001610516e-06},{\"date\":\"2025-05-01\",\"count\":236,\"percent\":1.2473156287802648e-05},{\"date\":\"2025-06-01\",\"count\":89,\"percent\":4.703859786501846e-06}]"' data-total-rows='6' data-total-columns='3' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["date", ["date", "date"]], ["count", ["integer", "i64"]], ["percent", ["number", "f64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>